<a href="https://colab.research.google.com/github/alessioborgi/DL_Project/blob/main/Source/InfoRetrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

==================================================

**Project Name:** Neural inverted index for fast and effective information retrieval\
**Course:** Deep Learning\
**University:** Sapienza Università di Roma

**Authors:**
  - [Alessio Borgi] (<tt>1952442</tt>)
  - [Eugenio Bugli] (<tt>1934824</tt>)
  - [Damiano Imola] (<tt>2109063</tt>)

**Date:** [November 2024 - Completion Date]


**Implementations**
*   Differentiable Search Index architecture
*   DSI-Multi dataset generation


**Novelties**
*   Dynamic pruning
*   Semantic and Stopwords Augmentation
*   Part Of Speech Masked Language Model (POS-MLM) Augmentation
*   Parameter Efficient Fine Tuning (PEFT) using LoRA, QLoRA, AdaLoRA, and ConvoLoRA adapters




==================================================

# 0: INSTALL & IMPORT LIBRARIES

In [ ]:
%%capture
!pip install -q --upgrade pip
# it takes quite a lot
!pip install -q pyserini==0.12.0
# !pip install -q pyserini==0.21.0
# !pip install -q faiss-gpu
!pip install -q pytorch-lightning transformers datasets torch wandb

In [ ]:
# base
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
from typing import List, Tuple
from tqdm import tqdm

# for pyserini stuffs
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# cool plots
import wandb

# torch
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# lightning
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor, StochasticWeightAveraging, DeviceStatsMonitor, ModelPruning


# HF and similar
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoModel, AutoTokenizer, AutoTokenizer, AutoModelForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, EncoderDecoderCache

# sklearn
from sklearn.preprocessing import normalize
from sklearn.cluster import AgglomerativeClustering, KMeans

# pyserini
# import faiss
from pyserini.index import IndexReader
from pyserini.search import SimpleSearcher
# from pyserini.search.lucene import LuceneSearcher

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer

# PEFT imports
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Currently using {device}")

Currently using cuda


In [ ]:
wandb.login()
# wandb.login(key="b3bce19a09c51bdf8a19eb3dc58f7c44de929e13") #(ALESSIO)
# wandb.login(key="6d550e12a1b8f716ebe580082f495c01ed2adf6c") #(DAMIANO)
wandb.login(key="551e67be2c716a42ea3230a7c4fc639fc985f98f") #(EUGENIO)
wandb.init(project="IR_DSI", resume="allow")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eugeniobugli15 (adavit). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1: LOAD TOKENIZED DATASET

In [ ]:
# copy to local memory
!cp '/content/drive/MyDrive/deep-learning-files/train_data_tokenized.pt' '/content/train_data_tokenized.pt'
!cp '/content/drive/MyDrive/deep-learning-files/validation_data_tokenized.pt' '/content/validation_data_tokenized.pt'
!cp '/content/drive/MyDrive/deep-learning-files/test_data_tokenized.pt' '/content/test_data_tokenized.pt'

In [ ]:
class DatasetLoader(torch.utils.data.Dataset):
    def __init__(self, file_name, up_to_k=5000):
        self.data = torch.load(file_name)[:up_to_k]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
train_dataset = DatasetLoader('/content/train_data_tokenized.pt', up_to_k=500)
val_dataset = DatasetLoader('/content/validation_data_tokenized.pt', up_to_k=100)
test_dataset = DatasetLoader('/content/test_data_tokenized.pt', up_to_k=100)

<ipython-input-7-01546771ad75>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data = torch.load(file_name)[:up_to_k]


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=3)
test_dataloader = DataLoader(test_dataset, batch_size=3)

In [ ]:
print(len(train_dataset), len(val_dataset), len(test_dataset))
print(len(train_dataset) + len(val_dataset) + len(test_dataset))

500 100 100
700


# 2: Original DSI Model

The encoder and decoder tokenizers require a max length, so to do not clip the input and use the whole input length.
The following numbers are not *magic* but are computed in the other file available in the repository names `InfoRetrievalDataset.ipynb`


In [ ]:
max_encoder_squence_len = 1797 # 48
max_decoder_squence_len = 4
max_decoder_squence_len_1000 = max_decoder_squence_len * 1000

In [ ]:
class DSIT5Model(pl.LightningModule):
    """DSI Base Model, as described in the original paper"""
    def __init__(self, model_name="t5-small", learning_rate=5e-5, max_decoder_squence_len=max_decoder_squence_len, max_decoder_squence_len_1000=max_decoder_squence_len_1000):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name) # transformer
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.learning_rate = learning_rate


    def training_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        # metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("train_loss", loss, on_epoch=True)
        return loss


    def validation_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("validation_loss", loss, on_epoch=True)
        return loss



    def test_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(queries, decoder_input_ids, decoder_1000_input_ids)

        self.log("test_loss", loss, on_epoch=True)
        return loss




    def compute_metrics(self, input_ids, decoder_input_ids, decoder_1000_input_ids):
        # infer top 1000 documents
        predicted_docids_tokenized = self.model.generate(input_ids, max_length=max_decoder_squence_len_1000)

        predicted_docids = []
        predicted_1000_docids = []
        for prediction in predicted_docids_tokenized:
            # retrieve top 1
            # decoded = self.tokenizer.decode(prediction[:max_decoder_squence_len], skip_special_tokens=True)

            decoded_top1 = self.tokenizer.decode(prediction[:max_decoder_squence_len], skip_special_tokens=True)
            splitted = decoded_top1.split()
            predicted_docids.append(splitted[0] if splitted else "")
            # predicted_docids.append(decoded.split()[0])

            # retrieve top 1000
            decoded_1000 = self.tokenizer.decode(prediction[:max_decoder_squence_len_1000], skip_special_tokens=True)
            predicted_1000_docids.append(decoded_1000.split())


        # ground truth
        target_docids = []
        for ground_truth in decoder_input_ids:
            decoded = self.tokenizer.decode(ground_truth, skip_special_tokens=True)
            target_docids.append(decoded)


        # compute metrics
        recall_at_1000 = self.compute_recall_at_1000(predicted_1000_docids, decoder_1000_input_ids)
        map = self.compute_map(predicted_docids, target_docids)

        return {
            "recall_at_1000": recall_at_1000,
            "map": map
        }




    def compute_recall_at_1000(self, predicted_docids, target_docids):
        recalls = []

        for predicted, target in zip(predicted_docids, target_docids):
            predicted_set = set(predicted[:1000])
            target_set = set(target)

            if not target_set:
                recalls.append(0)
                continue

            recall = len(predicted_set.intersection(target_set)) / len(target_set)
            recalls.append(recall)

        return np.mean(recalls)



    def compute_map(self, predicted_docids, target_docids):
        aps = []
        for predicted, target in zip(predicted_docids, target_docids):
            target_set = set(target.split())

            if not target_set:
                aps.append(0)
                continue

            precision_at_k = []
            num_hits = 0

            for i, doc in enumerate(predicted_docids):
                if doc in target_set:
                    num_hits += 1
                    precision_at_k.append(num_hits / (i + 1))

            # Average Precision for this query
            aps.append(np.mean(precision_at_k) if precision_at_k else 0)

        return np.mean(aps)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

## 2.1: Training

In [ ]:
model = DSIT5Model()

logger = WandbLogger(project="IR_DSI_Project")

# ===== CALLBACKS =====
checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='validation_loss', patience=3, mode='min')

# ===== DYNAMIC PRUNING ====
# removes individual weights based on magnitude or importance
# removes the ones with smallest L1 norm
# amount removes 20% of the smallest magnitude weights
pruning_callback = ModelPruning("l1_unstructured", amount=0.5)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback, pruning_callback],

    accelerator="auto",

    # gradient accumulation
    accumulate_grad_batches=4, # gradient is computed after 4 batches

    # mixed precision
    precision='16-mixed', # 16 bit precision of my model
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode
------------------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M | eval
------------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
0         Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▃▃▃▆▆▆███
train_loss_epoch,▁█▇▇
train_loss_step,█▁▁
trainer/global_step,▁▁▁▃▃▄▆▆▇██
validation_loss,▁█▇▇
epoch,3
train_loss_epoch,20.75
train_loss_step,20.75
trainer/global_step,167
validation_loss,20.75


In [ ]:
# save locally
torch.save(model, '/content/checkpoints/<CHECKPOINT_NAME>')
# save to GDrive
!cp '/content/model.pth' '/content/drive/MyDrive/deep-learning-files/checkpoint_<EPOCH>.pth'


# save locally
torch.save(model, '/content/model.pth')
# save to GDrive
!cp '/content/model.pth' '/content/drive/MyDrive/deep-learning-files/model_<EPOCH>.pth'


# save locally
torch.save(model.state_dict(), '/content/model_state_dict.pth')
# save to GDrive
!cp '/content/model_state_dict.pth' '/content/drive/MyDrive/deep-learning-files/model_state_dict_<EPOCH>.pth'

# 3: PEFT (Parameter Efficient Finetuning)

Fine-tuning large T5 models from scratch can be expensive in both time and GPU memory. To alleviate this, we
decided to employ variour Low-Rank adapters:

1.   LoRA
2.   QLoRA
3.   AdaLoRA
4.   ConvoLoRA

NOTE: in the following code we highlighted the major differences in the base lightning class embracing them with lines containing some equals (`=`)



## 3.1: LoRA

LoRA freezes the original T5 weights
and injects small low-rank adapter modules into the attention layers.

During training, only these adapter weights are
updated—dramatically cutting the total trainable parameter count and thus reducing memory needs and speeding up
convergence.

In [ ]:
class DSIT5ModelLORA(pl.LightningModule):
    def __init__(self, model_name="t5-small", learning_rate=5e-5, max_decoder_squence_len=max_decoder_squence_len, max_decoder_squence_len_1000=max_decoder_squence_len_1000, lora_r=8, lora_alpha=32, lora_dropout=0.1):
        super().__init__()

        self.T5 = T5ForConditionalGeneration.from_pretrained(model_name) # transformer

        # ===========================================================================
        # PEFT sets requires_grad=False on all original T5 layers
        # LoRA parameters (the low-rank adapters) are injected into the T5 attention layers and do requires_grad=True
        # during training, only these small LoRA adapters get updated, leaving the main T5 weights untouched
        self.peft_config = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            inference_mode=False,
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout
        )
        # ===========================================================================

        self.model = get_peft_model(self.T5, self.peft_config)

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.learning_rate = learning_rate


    def training_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        # metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("train_loss", loss, on_epoch=True)
        return loss


    def validation_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("validation_loss", loss, on_epoch=True)
        return loss



    def test_step(self, batch, batch_idx):
        queries = torch.Tensor(batch['query']).squeeze(1).to(device)
        input_ids = torch.Tensor(batch['input_ids']).squeeze(1)
        decoder_input_ids = torch.Tensor(batch['decoder_input_ids']).squeeze(1)
        decoder_1000_input_ids = torch.Tensor(batch['decoder_ranked_input_ids']).squeeze(1)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(queries, decoder_input_ids, decoder_1000_input_ids)

        self.log("test_loss", loss, on_epoch=True)
        return loss



    def compute_metrics(self, input_ids, decoder_input_ids, decoder_1000_input_ids):
        # get_base_model().generate(...) uses the LoRA adapters in the forward pass
        # We don’t lose the LoRA modifications by calling the base model, because LoRA modifies the internals of T5’s attention layers

        # infer top 1000 documents
        # ===========================================================================
        predicted_docids_tokenized = self.model.get_base_model().generate(input_ids, max_length=max_decoder_squence_len_1000)
        # ===========================================================================

        predicted_docids = []
        predicted_1000_docids = []
        for prediction in predicted_docids_tokenized:

            # retrieve top 1
            # decoded = self.tokenizer.decode(prediction[:max_decoder_squence_len], skip_special_tokens=True)

            decoded_top1 = self.tokenizer.decode(prediction[:max_decoder_squence_len], skip_special_tokens=True)
            splitted = decoded_top1.split()
            predicted_docids.append(splitted[0] if splitted else "")
            # predicted_docids.append(decoded.split()[0])

            # retrieve top 1000
            decoded_1000 = self.tokenizer.decode(prediction[:max_decoder_squence_len_1000], skip_special_tokens=True)
            predicted_1000_docids.append(decoded_1000.split())

        # ground truth
        target_docids = []
        for ground_truth in decoder_input_ids:
            decoded = self.tokenizer.decode(ground_truth, skip_special_tokens=True)
            target_docids.append(decoded)


        # compute metrics
        recall_at_1000 = self.compute_recall_at_1000(predicted_1000_docids, decoder_1000_input_ids)
        map = self.compute_map(predicted_docids, target_docids)

        return {
            "recall_at_1000": recall_at_1000,
            "map": map
        }




    def compute_recall_at_1000(self, predicted_docids, target_docids):
        recalls = []

        for predicted, target in zip(predicted_docids, target_docids):
            predicted_set = set(predicted[:1000])
            target_set = set(target)

            if not target_set:
                recalls.append(0)
                continue

            recall = len(predicted_set.intersection(target_set)) / len(target_set)
            recalls.append(recall)

        return np.mean(recalls)



    def compute_map(self, predicted_docids, target_docids):
        aps = []
        for predicted, target in zip(predicted_docids, target_docids):
            target_set = set(target.split())

            if not target_set:
                aps.append(0)
                continue

            precision_at_k = []
            num_hits = 0

            for i, doc in enumerate(predicted_docids):
                if doc in target_set:
                    num_hits += 1
                    precision_at_k.append(num_hits / (i + 1))

            # Average Precision for this query
            aps.append(np.mean(precision_at_k) if precision_at_k else 0)

        return np.mean(aps)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

In [ ]:
model = DSIT5ModelLORA(
    lora_r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

logger = WandbLogger(project="IR_DSI_Project")

# ===== CALLBACKS =====
checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-lora-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='validation_loss', patience=3, mode='min')

# ===== DYNAMIC PRUNING ====
# removes individual weights based on magnitude or importance
# removes the ones with smallest L1 norm
# amount removes 20% of the smallest magnitude weights
pruning_callback = ModelPruning("l1_unstructured", amount=0.5)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],#pruning_callback],

    accelerator="auto",

    # gradient accumulation
    accumulate_grad_batches=4, # gradient is computed after 4 batches

    # mixed precision
    precision='16-mixed', # 16 bit precision of my model
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params | Mode 
-------------------------------------------------------------
0 | T5    | T5ForConditionalGeneration | 60.8 M | eval 
1 | model | PeftModelForSeq2SeqLM      | 60.8 M | train
-------------------------------------------------------------
294 K     Trainable params
60.5 M    Non-trainable params
60.8 M    Total params
243.206   Total estimated model params size (MB)
362       Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▁▁▁▁▁▃▃▆▆▁▁▁▃▃▆▆██
train_loss_epoch,▁▁▁▁█▆▁▁▁▁
trainer/global_step,▁▁▁▁▁▁▁▁▃▃▆▆▁▁▁▃▃▆▆██
validation_loss,▁▁▁▁█▇▁▁▁▁▁
epoch,3
train_loss_epoch,12.58074
trainer/global_step,3
validation_loss,13.29458


## 3.2: QLoRA

Although LoRA alone is parameter-efficient, large T5 models can still require substantial GPU memory. We decide
therefore to also provide for an alternative model version (in addition to the basic one and to the LORA version), to
further minimize memory consumption, we apply QLoRA, which additionally quantizes the T5 weights to a 4-bit
format using bitsandbytes library.

In this setup, the base T5 is compressed into 4-bit precision, while LoRA’s low-rank
adapters remain at higher precision for effective fine-tuning. This model, further reduce memory requirements and
training overhead without sacrificing retrieval accuracy.

In [ ]:
%%capture
!pip uninstall -y bitsandbytes accelerate peft

Found existing installation: bitsandbytes 0.45.1
Uninstalling bitsandbytes-0.45.1:
  Successfully uninstalled bitsandbytes-0.45.1
Found existing installation: accelerate 1.3.0
Uninstalling accelerate-1.3.0:
  Successfully uninstalled accelerate-1.3.0
Found existing installation: peft 0.14.0
Uninstalling peft-0.14.0:
  Successfully uninstalled peft-0.14.0


In [ ]:
%%capture
!python -m bitsandbytes

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++++++++++ OTHER +++++++++++++++++++++++++++
CUDA specs: CUDASpecs(highest_compute_capability=(7, 5), cuda_version_string='121', cuda_version_tuple=(12, 1))
PyTorch settings found: CUDA_VERSION=121, Highest Compute Capability: (7, 5).
To manually override the PyTorch CUDA version please see: https://github.com/TimDettmers/bitsandbytes/blob/main/docs/source/nonpytorchcuda.mdx
The directory listed in your path is found to be non-existent: /sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events
The directory listed in your path is found to be non-existent: //172.28.0.1
The directory listed in your path is found to be non-existent: 8013
The directory listed in your path is found to be non-existent: //colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/

In [ ]:
%%capture
!pip install bitsandbytes -U -q
!pip install accelerate peft -q
!pip install -U gdown transformers -q

In [ ]:
#!pip install bitsandbytes -q -U
#!pip install bitsandbytes==0.39.1
#!pip install bitsandbytes

import transformers as tra
import peft as pf
import bitsandbytes as bnb

from transformers import T5ForConditionalGeneration, T5Tokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from bitsandbytes.optim import AdamW8bit

print(pf.__version__)
print(tra.__version__)
print(bnb.__version__)

0.14.0
4.48.1
0.45.1


In [ ]:
class DSIT5ModelQLORA(pl.LightningModule):
    def __init__(self, model_name="t5-small", learning_rate=5e-5, max_decoder_squence_len=max_decoder_squence_len, max_decoder_squence_len_1000=max_decoder_squence_len_1000, lora_r=8, lora_alpha=32, lora_dropout=0.1):
        super().__init__()

        # ===========================================================================
        # 4-bit quantization
        bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.float16)

        # load T5 in 4-bit precision
        self.base_t5 = T5ForConditionalGeneration.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

        self.peft_config = LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            inference_mode=False,
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout
        )
        # ===========================================================================

        self.model = get_peft_model(self.base_t5, self.peft_config)

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)

        self.learning_rate = learning_rate
        self.max_decoder_squence_len = max_decoder_squence_len
        self.max_decoder_squence_len_1000 = max_decoder_squence_len_1000

    def training_step(self, batch, batch_idx):
        # Expecting your batch to have these keys
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("validation_loss", loss, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(queries, decoder_input_ids, decoder_1000_input_ids)

        self.log("test_loss", loss, on_epoch=True)
        return loss

    def compute_metrics(self, input_ids, decoder_input_ids, decoder_1000_input_ids):
        # ===========================================================================
        # For generation, we call get_base_model() to access T5ForConditionalGeneration.generate(...)
        predicted_docids_tokenized = self.model.get_base_model().generate(input_ids=input_ids, max_length=self.max_decoder_squence_len_1000)
        # ===========================================================================

        predicted_docids = []
        predicted_1000_docids = []

        for prediction in predicted_docids_tokenized:
            # top 1
            decoded_top1 = self.tokenizer.decode(prediction[:self.max_decoder_squence_len], skip_special_tokens=True)
            splitted = decoded_top1.split()
            predicted_docids.append(splitted[0] if splitted else "")

            # top 1000
            decoded_1000 = self.tokenizer.decode(prediction[:self.max_decoder_squence_len_1000], skip_special_tokens=True)
            predicted_1000_docids.append(decoded_1000.split())

        # ground truth
        target_docids = []
        for ground_truth in decoder_input_ids:
            decoded = self.tokenizer.decode(ground_truth, skip_special_tokens=True)
            target_docids.append(decoded)

        # compute metrics
        recall_at_1000 = self.compute_recall_at_1000(predicted_1000_docids, decoder_1000_input_ids)
        map_val = self.compute_map(predicted_docids, target_docids)

        return {
            "recall_at_1000": recall_at_1000,
            "map": map_val
        }

    def compute_recall_at_1000(self, predicted_docids, target_docids):
        recalls = []
        for predicted, target in zip(predicted_docids, target_docids):
            predicted_set = set(predicted[:1000])
            target_set = set(target)
            if not target_set:
                recalls.append(0)
                continue
            recall = len(predicted_set.intersection(target_set)) / len(target_set)
            recalls.append(recall)
        return np.mean(recalls)

    def compute_map(self, predicted_docids, target_docids):
        aps = []
        for predicted, target in zip(predicted_docids, target_docids):
            target_set = set(target.split())
            if not target_set:
                aps.append(0)
                continue

            precision_at_k = []
            num_hits = 0
            for i, doc in enumerate(predicted):
                if doc in target_set:
                    num_hits += 1
                    precision_at_k.append(num_hits / (i + 1))

            avg_precision = np.mean(precision_at_k) if precision_at_k else 0
            aps.append(avg_precision)

        return np.mean(aps)

    def configure_optimizers(self):
        optimizer = AdamW8bit(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
model = DSIT5ModelQLORA(
    lora_r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

logger = WandbLogger(project="IR_DSI_Project")

# ===== CALLBACKS =====
checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-lora-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='validation_loss', patience=3, mode='min')

# ===== DYNAMIC PRUNING ====
# removes individual weights based on magnitude or importance
# removes the ones with smallest L1 norm
# amount removes 20% of the smallest magnitude weights
pruning_callback = ModelPruning("l1_unstructured", amount=0.5)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback], #pruning_callback],

    accelerator="auto",

    # gradient accumulation
    accumulate_grad_batches=4, # gradient is computed after 4 batches

    # mixed precision
    precision='16-mixed', # 16 bit precision of my model
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
INFO:pytorch_lightni

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▃▃▆▆██
train_loss_epoch,█▁▆▁
trainer/global_step,▁▁▃▃▆▆██
validation_loss,▁▁▁▁
epoch,3
train_loss_epoch,13.39073
trainer/global_step,3
validation_loss,13.63606


## 3.3: AdaLoRA

n addition to the original LoRA approach and its quantized version, we explored Adaptive LoRA (AdaLoRA). The fundamental idea behind AdaLoRA is to dynamically adjust the effective rank of the LoRA
adapters throughout training.

By starting with a higher initial rank (init_r), the model first gains sufficient capacity
to represent the task. After a specified warm-up period, AdaLoRA gradually prunes the less important dimensions
in the low-rank matrices, converging to a smaller final rank (target_r).

In [ ]:
from peft import (
    AdaLoraConfig,
    get_peft_model,
    TaskType
)
class DSIT5ModelAdaLORA(pl.LightningModule):
    def __init__(
        self,
        model_name="t5-small",
        learning_rate=5e-5,
        max_decoder_squence_len=max_decoder_squence_len,
        max_decoder_squence_len_1000=max_decoder_squence_len_1000,
        init_r=12, # initial rank for lora adapters
        target_r=4, # final desired rank for lora adapters
        lora_alpha=32, lora_dropout=0.1,
        init_steps=20, target_steps=200, # steps after which rank update begins and ends
        delta_steps=10 # interval for each update
    ):
        super().__init__()

        self.base_t5 = T5ForConditionalGeneration.from_pretrained(model_name)


        # ===========================================================================
        self.adalora_config = AdaLoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            init_r = init_r,
            target_r = target_r,
            tinit = init_steps,
            tfinal = target_steps,
            deltaT = delta_steps,
            lora_alpha = lora_alpha,
            lora_dropout = lora_dropout
        )

        self.model = get_peft_model(self.base_t5, self.adalora_config)
        # ===========================================================================

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.learning_rate = learning_rate
        self.max_decoder_squence_len = max_decoder_squence_len
        self.max_decoder_squence_len_1000 = max_decoder_squence_len_1000

    def training_step(self, batch, batch_idx):
        # 'query', 'input_ids', 'decoder_input_ids', 'decoder_ranked_input_ids'
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        # indexing task
        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        # retrieval task
        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss
        metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("validation_loss", loss, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        index_output = self.model(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        retrieval_output = self.model(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss
        metrics = self.compute_metrics(queries, decoder_input_ids, decoder_1000_input_ids)

        self.log("test_loss", loss, on_epoch=True)
        return loss

    def compute_metrics(self, input_ids, decoder_input_ids, decoder_1000_input_ids):
        # ===========================================================================
        # Use the T5 "base model" so we can call generate(...)
        predicted_docids_tokenized = self.model.get_base_model().generate(
            input_ids=input_ids,
            max_length=self.max_decoder_squence_len_1000
        )
        # ===========================================================================

        predicted_docids = []
        predicted_1000_docids = []

        for prediction in predicted_docids_tokenized:
            # top 1
            decoded_top1 = self.tokenizer.decode(
                prediction[:self.max_decoder_squence_len],
                skip_special_tokens=True
            )
            splitted = decoded_top1.split()
            predicted_docids.append(splitted[0] if splitted else "")

            # top 1000
            decoded_1000 = self.tokenizer.decode(
                prediction[:self.max_decoder_squence_len_1000],
                skip_special_tokens=True
            )
            predicted_1000_docids.append(decoded_1000.split())

        # ground truth
        target_docids = []
        for gt in decoder_input_ids:
            decoded = self.tokenizer.decode(gt, skip_special_tokens=True)
            target_docids.append(decoded)

        # compute recall@1000
        recall_at_1000 = self.compute_recall_at_1000(predicted_1000_docids, decoder_1000_input_ids)
        map_val = self.compute_map(predicted_docids, target_docids)

        return {
            "recall_at_1000": recall_at_1000,
            "map": map_val
        }

    def compute_recall_at_1000(self, predicted_docids, target_docids):
        recalls = []
        for predicted, target in zip(predicted_docids, target_docids):
            predicted_set = set(predicted[:1000])
            target_set = set(target)
            if not target_set:
                recalls.append(0)
                continue

            recall = len(predicted_set.intersection(target_set)) / len(target_set)
            recalls.append(recall)
        return np.mean(recalls)

    def compute_map(self, predicted_docids, target_docids):
        aps = []
        for predicted, target in zip(predicted_docids, target_docids):
            target_set = set(target.split())
            if not target_set:
                aps.append(0)
                continue

            precision_at_k = []
            num_hits = 0
            for i, doc in enumerate(predicted):
                if doc in target_set:
                    num_hits += 1
                    precision_at_k.append(num_hits / (i + 1))

            avg_precision = np.mean(precision_at_k) if precision_at_k else 0
            aps.append(avg_precision)

        return np.mean(aps)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

In [ ]:
model = DSIT5ModelAdaLORA()

logger = WandbLogger(project="IR_DSI_Project")

# ===== CALLBACKS =====
checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-lora-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='validation_loss', patience=3, mode='min')

pruning_callback = ModelPruning("l1_unstructured", amount=0.5)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback], #pruning_callback],

    accelerator="auto",

    # gradient accumulation
    accumulate_grad_batches=4, # gradient is computed after 4 batches

    # mixed precision
    precision='16-mixed', # 16 bit precision of my model
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type                       | Params | Mode 
---------------------------------------------------------------
0 | base_t5 | T5ForConditionalGeneration | 62.1 M | eval 
1 | model   | PeftModelForSeq2SeqLM      | 62.1 M | train
---------------------------------------------------------------
1.6 M     Trainable params
60.5 M    Non-trainable params
62.1 M    Total params
248.520   Total estimated model params size (MB)
962       Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▃▃▆▆██
train_loss_epoch,▃▄▁█
trainer/global_step,▁▁▃▃▆▆██
validation_loss,▁▁▁▁
epoch,3
train_loss_epoch,15.07547
trainer/global_step,3
validation_loss,15.78931


## 3.4: ConvoLoRA

Another extension to LoRA that we investigated is ConvLoRA, sometimes referred to as LoCon. This method adds a
lightweight convolutional operation to the LoRA adaptation, originally developed and thought for image-generation
models (e.g., Stable Diffusion).

In the NLP setting, we introduce a small depthwise convolution layer that operates
4Running Title for Header across the sequence dimension, followed by a low-rank linear mapping. Finally, a scaling factor (alpha) and a residual
connection are applied, analogous to standard LoRA.

The following block is needed to implement the ConvoLoRA adapter.

In [ ]:
class ConvoAdapter(nn.Module):
    def __init__(
        self,
        hidden_dim,
        conv_kernel_size=3,
        rank=4,
        alpha=32,
        dropout=0.1
    ):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.rank = rank
        self.alpha = alpha
        self.dropout = nn.Dropout(dropout)

        self.conv = nn.Conv1d(
            in_channels=hidden_dim,
            out_channels=hidden_dim,
            kernel_size=conv_kernel_size,
            padding=(conv_kernel_size - 1) // 2,
            groups=hidden_dim  # depthwise
        )

        # Low-rank linear mapping
        self.down_proj = nn.Linear(hidden_dim, rank, bias=False)
        self.up_proj   = nn.Linear(rank, hidden_dim, bias=False)

    def forward(self, hidden_states):
        # hidden_states shape: [batch_size, seq_len, hidden_dim]
        bsz, seq_len, dim = hidden_states.shape
        assert dim == self.hidden_dim, "Mismatched hidden dim in LoConAdapter"
        hs = hidden_states.transpose(1, 2)
        conv_out = self.conv(hs)
        conv_out = conv_out.transpose(1, 2)
        conv_out = self.dropout(conv_out)
        down = self.down_proj(conv_out)
        up   = self.up_proj(down)
        scaled = up * self.alpha
        return hidden_states + scaled

In [ ]:
class DSIT5ModelConvoLORA(pl.LightningModule):
    def __init__(
        self,
        model_name="t5-small",
        learning_rate=5e-5,
        max_decoder_squence_len=max_decoder_squence_len,
        max_decoder_squence_len_1000=max_decoder_squence_len_1000,
        conv_kernel_size=3,
        conv_lora_rank=4,
        conv_lora_alpha=32,
        conv_lora_dropout=0.1
    ):
        super().__init__()
        self.save_hyperparameters()

        self.base_t5 = T5ForConditionalGeneration.from_pretrained(model_name)

        # ===========================================================================
        for param in self.base_t5.parameters():
            param.requires_grad = False

        ff_sublayer = self.base_t5.encoder.block[-1].layer[1]

        ff_sublayer.lo_con = ConvoAdapter(
            hidden_dim=self.base_t5.config.d_model,
            conv_kernel_size=conv_kernel_size,
            rank=conv_lora_rank,
            alpha=conv_lora_alpha,
            dropout=conv_lora_dropout
        )
        # ===========================================================================

        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.learning_rate = learning_rate
        self.max_decoder_squence_len = max_decoder_squence_len
        self.max_decoder_squence_len_1000 = max_decoder_squence_len_1000
        self.model = self.base_t5

    def forward(self, input_ids, attention_mask=None, labels=None):
        encoder_outputs = self.model.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        last_hidden = encoder_outputs.last_hidden_state
        ff_sublayer = self.model.encoder.block[-1].layer[1]
        if hasattr(ff_sublayer, "lo_con"):
            last_hidden = ff_sublayer.lo_con(last_hidden)
        new_enc_out = [last_hidden]
        outputs = self.model(
            encoder_outputs=new_enc_out,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True
        )
        return outputs

    def training_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        index_output = self(
            input_ids=input_ids,
            labels=decoder_input_ids
        )
        index_loss = index_output.loss
        retrieval_output = self(
            input_ids=input_ids,
            labels=queries
        )
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss
        self.log("train_loss", loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)

        index_output = self(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss

        retrieval_output = self(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(input_ids, decoder_input_ids, decoder_1000_input_ids)

        self.log("validation_loss", loss, on_epoch=True)
        return loss

    def test_step(self, batch, batch_idx):
        queries = batch['query'].squeeze(1).to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        decoder_input_ids = batch['decoder_input_ids'].squeeze(1).to(device)
        decoder_1000_input_ids = batch['decoder_ranked_input_ids'].squeeze(1).to(device)
        index_output = self(input_ids=input_ids, labels=decoder_input_ids)
        index_loss = index_output.loss
        retrieval_output = self(input_ids=input_ids, labels=queries)
        retrieval_loss = retrieval_output.loss

        loss = index_loss + retrieval_loss

        metrics = self.compute_metrics(queries, decoder_input_ids, decoder_1000_input_ids)

        self.log("test_loss", loss, on_epoch=True)
        return loss

    def compute_metrics(self, input_ids, decoder_input_ids, decoder_1000_input_ids):
        predicted_docids_tokenized = self.model.generate(
            input_ids=input_ids,
            max_length=self.max_decoder_squence_len_1000
        )

        predicted_docids = []
        predicted_1000_docids = []

        for prediction in predicted_docids_tokenized:
            decoded_top1 = self.tokenizer.decode(
                prediction[:self.max_decoder_squence_len],
                skip_special_tokens=True
            )
            splitted = decoded_top1.split()
            predicted_docids.append(splitted[0] if splitted else "")
            decoded_1000 = self.tokenizer.decode(
                prediction[:self.max_decoder_squence_len_1000],
                skip_special_tokens=True
            )
            predicted_1000_docids.append(decoded_1000.split())
        target_docids = []
        for gt in decoder_input_ids:
            decoded = self.tokenizer.decode(gt, skip_special_tokens=True)
            target_docids.append(decoded)
        recall_at_1000 = self.compute_recall_at_1000(predicted_1000_docids, decoder_1000_input_ids)
        map_val = self.compute_map(predicted_docids, target_docids)

        return {
            "recall_at_1000": recall_at_1000,
            "map": map_val
        }

    def compute_recall_at_1000(self, predicted_docids, target_docids):
        recalls = []
        for predicted, target in zip(predicted_docids, target_docids):
            predicted_set = set(predicted[:1000])
            target_set = set(target)
            if not target_set:
                recalls.append(0)
                continue

            recall = len(predicted_set.intersection(target_set)) / len(target_set)
            recalls.append(recall)
        return np.mean(recalls)

    def compute_map(self, predicted_docids, target_docids):
        aps = []
        for predicted, target in zip(predicted_docids, target_docids):
            target_set = set(target.split())
            if not target_set:
                aps.append(0)
                continue

            precision_at_k = []
            num_hits = 0
            for i, doc in enumerate(predicted):
                if doc in target_set:
                    num_hits += 1
                    precision_at_k.append(num_hits / (i + 1))

            avg_precision = np.mean(precision_at_k) if precision_at_k else 0
            aps.append(avg_precision)

        return np.mean(aps)

    def configure_optimizers(self):
        trainable_params = [p for p in self.parameters() if p.requires_grad]
        optimizer = torch.optim.AdamW(trainable_params, lr=self.learning_rate)
        return optimizer

In [ ]:
model = DSIT5ModelConvoLORA()

logger = WandbLogger(project="IR_DSI_Project")

# ===== CALLBACKS =====
checkpoint_callback = ModelCheckpoint(
    monitor='validation_loss',
    dirpath='checkpoints/',
    filename='dsi-t5-lora-{epoch:02d}-{val_loss:.2f}',
    save_top_k=1,
    mode='min'
)

early_stopping_callback = EarlyStopping(monitor='validation_loss', patience=3, mode='min')

pruning_callback = ModelPruning("l1_unstructured", amount=0.5)

trainer = pl.Trainer(
    max_epochs=5,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback], #pruning_callback],

    accelerator="auto",

    # gradient accumulation
    accumulate_grad_batches=4, # gradient is computed after 4 batches

    # mixed precision
    precision='16-mixed', # 16 bit precision of my model
)

# Train the model
trainer.fit(model, train_dataloader, val_dataloaders=val_dataloader)

wandb.finish()

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type                       | Params | Mode
--------------------------------------------------------------
0 | base_t5 | T5ForConditionalGeneration | 60.5 M | eval
--------------------------------------------------------------
6.1 K     Trainable params
60.5 M    Non-trainable params
60.5 M    Total params
242.051   Total estimated model params size (MB)
5         Modules in train mode
277       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▃▃▆▆██
train_loss_epoch,▁█▃▆
trainer/global_step,▁▁▃▃▆▆██
validation_loss,▁▁▁▁
epoch,3
train_loss_epoch,38.40944
trainer/global_step,3
validation_loss,36.55384
